In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from tqdm.notebook import tqdm

import os

C:\Users\jose.ggarza\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df_data = pd.read_csv("../Dataset/processedDataWithThaliaAdded.csv",nrows=10000,delimiter='|')
df_data.head(20)

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,False
1,32836,#famous,#famous,famous,Jilly Gagnon,Jilly Gagnon,"jilly gagnon, todd strasser",de,", Englisch",2.0,...,HarperCollins Publishers Inc,Planet!,2018-02-13,Eine turbulente Social-Media-Liebesgeschichte ...,YFM,YXQ,http://books.google.com/books/content?id=kDM5D...,NOT_MATURE,368.0,True
2,38332,''Grounded'' at Fort Irwin,''Grounded'' at Fort Irwin,grounded at fort irwin,Keon Fleurima,Keon Fleurima,keon fleurima,un,Englisch,2.0,...,Xlibris,Xlibris Corporation,2011-11-15,Grounded is about me getting in trouble. After...,YNMH,YFB,http://books.google.com/books/content?id=wY-DA...,NOT_MATURE,NaN,False
3,51507,''It's Not Ok'',''It's Not OK'',its not ok,Amy Castle,Amy Castle,amy castle,en,Englisch,2.0,...,Xlibris,Xlibris Corporation,2013-11-11,"Although it's been around forever, bullying ha...",YFB,YXW,http://books.google.com/books/content?id=B8CDA...,NOT_MATURE,32.0,True
4,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,NaN,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,NaN,2008,NaN,YFM,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,False
5,48465,'...startled by his furry shorts!',Startled by His Furry Shorts,startled by his furry shorts,Louise Rennison,Louise Rennison,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,HarperCollins UK,2007,On the rack of romance. And also in the oven o...,YFM,5AN,http://books.google.com/books/content?id=I_2fl...,NOT_MATURE,271.0,False
6,71057,'Acaso Todos Se Pedorrean? (Does Everybody Fart?),åÀAcaso Todos Se Pedorrean? (Does Everybody Fa...,acaso todos se pedorrean does everybody fart,Luis Rodriguez,Luis Rodriguez,luis rodriguez,es,Spanisch,2.0,...,Luis fernando rodriguez,Lulu.com,2017-05,Un frijol curioso va en una olorosa aventura a...,YFB,,http://books.google.com/books/content?id=A80wD...,NOT_MATURE,NaN,False
7,32319,'Dancing in my nuddy-pants!',Dancing in My Nuddy-pants,dancing in my nuddy pants,Louise Rennison,Louise Rennison,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,NaN,2006,The further confessions of teenaged Georgia Ni...,YFM,5AM,NaN,NOT_MATURE,223.0,False
8,29592,'Existen las hadas?,¿existen Las Hadas?,existen las hadas,Roser Rovira,Roser Rovira,roser rovira,un,Spanisch,2.0,...,Blurb,Blurb,2018-11-27,¿Existe la magia? Mamá cree que todo es posibl...,YFB,,http://books.google.com/books/content?id=D2lpv...,NOT_MATURE,24.0,False
9,38212,'Knocked out by my nunga-nungas.',Knocked Out by My Nunga-Nungas,knocked out by my nunga nungas,Louise Rennison,Louise Rennison,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,Harper Collins,2009-10-06,As I was going out of my bedroom door I rememb...,YFM,5AM,http://books.google.com/books/content?id=B4caU...,NOT_MATURE,208.0,False


In [3]:
df_dataset = df_data[['itemID','title_thalia','author_thalia','publisher_api','main topic','subtopics']]
df_dataset.fillna("",inplace=True)

df_dataset['Metadata'] = df_dataset[['title_thalia','author_thalia','publisher_api','main topic','subtopics']].apply(lambda x: ' '.join(x), axis = 1)
df_dataset['Topic'] = df_dataset[['main topic','subtopics']].apply(lambda x: ' '.join(x), axis = 1)


#df_dataset['MainTpoic/SubTopic'] = df_dataset['MainTpoic/SubTopic'].str.replace('|',' ')
#df_dataset.head(20)

C:\Users\jose.ggarza\Anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-3-bfa55cf4083f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset['Metadata'] = df_dataset[['title_thalia','author_thalia','publisher_api','main topic','subtopics']].apply(lambda x: ' '.join(x), axis = 1)
<ipython-input-3-bfa55cf4083f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [5]:
book_search = 'louise rennison'
display(df_dataset[df_dataset.author_thalia==book_search])

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,Metadata,Topic
4,13238,then he ate my boy entrancers,louise rennison,,YFM,4Z GB ACN YFQ,then he ate my boy entrancers louise rennison...,YFM 4Z GB ACN YFQ
5,48465,startled by his furry shorts,louise rennison,HarperCollins UK,YFM,5AN,startled by his furry shorts louise rennison H...,YFM 5AN
7,32319,dancing in my nuddy pants,louise rennison,,YFM,5AM,dancing in my nuddy pants louise rennison YFM...,YFM 5AM
9,38212,knocked out by my nunga nungas,louise rennison,Harper Collins,YFM,5AM,knocked out by my nunga nungas louise rennison...,YFM 5AM
10,18472,luuurve is a many trousered thing,louise rennison,,YFM,5AM,luuurve is a many trousered thing louise renni...,YFM 5AM
18,21391,stop in the name of pants,louise rennison,,YFM,5AM YFQ,stop in the name of pants louise rennison YFM...,YFM 5AM YFQ
903,17850,a midsummer tights dream,louise rennison,HarperCollins Children's Books,YFM,YFQ,a midsummer tights dream louise rennison Harpe...,YFM YFQ


In [6]:
indices =pd.Series(df_dataset.index,index = df_dataset["title_thalia"]).drop_duplicates()
indices

title_thalia
 tripion  minaccia dallo spazio                                       0
famous                                                                1
grounded at fort irwin                                                2
its not ok                                                            3
 then he ate my boy entrancers                                        4
                                                                   ... 
aladdin [with cd]                                                  1995
aladdin and his magic lamp from the one thousand and one nights    1996
aladdin and other tales from the arabian nights                    1997
aladdin and the magic lamp                                         1998
aladdin and the magic lamp classic fiction                         1999
Length: 2000, dtype: int64

In [7]:
#Recommendations based on Metadata (Topic/Subtopic)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_dataset['Topic'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [8]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation = linear_kernel(tfidf_matrix, tfidf_matrix)



In [9]:
#Recommendations based on Metadata (Topic/Subtopic)
def get_recommendations_topic(title, cosine_sim, method_name):
 # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the top 5 most similar books
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:5]
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    # Return the top 5 most similar book
    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values),'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'Topic': list(df_dataset['Topic'].iloc[book_indices].values), method_name: book_pred_score})

In [10]:
book_search = 'luuurve is a many trousered thing'
print('Recommendation for book:')
display(df_dataset[df_dataset.title_thalia==book_search])
print('Content-based RecSyS based on Main Topic/Subtopic')
get_recommendations_topic(book_search,content_correlation, 'Similarity')

Recommendation for book:


,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,Metadata,Topic
10,18472,luuurve is a many trousered thing,louise rennison,,YFM,5AM,luuurve is a many trousered thing louise renni...,YFM 5AM


Content-based RecSyS based on Main Topic/Subtopic


,itemID,title_thalia,Topic,Similarity
0,32319,dancing in my nuddy pants,YFM 5AM,1.000000
1,38212,knocked out by my nunga nungas,YFM 5AM,1.000000
2,21391,stop in the name of pants,YFM 5AM YFQ,0.878847
3,26209,access all awkward,YFM 5AM YFS,0.830500
4,43871,a bad boy can be good for a girl,YFM,0.532365


In [11]:
# Recommendations Based of Author
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix2 = tfidf.fit_transform(df_dataset['author_thalia'])
tfidf_df2 = pd.DataFrame(tfidf_matrix2.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count2.fit_transform(df_dataset['author_thalia'])

#count_matrix

In [12]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)

In [13]:
#Recommendations based on Author
def get_recommendations_author(title, cosine_sim, method_name):
   # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwise similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the top 5 most similar books
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:5]
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    return pd.DataFrame({'itemID': list(df_dataset['itemID'].iloc[book_indices].values),'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'author_thalia': list(df_dataset['author_thalia'].iloc[book_indices].values), method_name: book_pred_score})

In [14]:
book_search = 'luuurve is a many trousered thing'
print('Recommendation for book:')
display(df_dataset[df_dataset.title_thalia==book_search])
print('Content-based RecSyS based on Author')
get_recommendations_author(book_search,content_correlation2, 'Predicted Rating')

Recommendation for book:


,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,Metadata,Topic
10,18472,luuurve is a many trousered thing,louise rennison,,YFM,5AM,luuurve is a many trousered thing louise renni...,YFM 5AM


Content-based RecSyS based on Author


,itemID,title_thalia,author_thalia,Predicted Rating
0,13238,then he ate my boy entrancers,louise rennison,1.0
1,48465,startled by his furry shorts,louise rennison,1.0
2,32319,dancing in my nuddy pants,louise rennison,1.0
3,38212,knocked out by my nunga nungas,louise rennison,1.0
4,21391,stop in the name of pants,louise rennison,1.0


In [15]:
# Recommendations Based of Publisher
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix3 = tfidf.fit_transform(df_dataset['publisher_api'])
tfidf_df3 = pd.DataFrame(tfidf_matrix3.toarray(), index=df_dataset.index.tolist())

#count3=CountVectorizer(stop_words="english")
#count_matrix3= count3.fit_transform(df_dataset['publisher_api'])



In [16]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation3 = linear_kernel(tfidf_matrix3, tfidf_matrix3)

In [17]:
def get_recommendations_publisher(title, cosine_sim, method_name):
    # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the top 5 most similar movies
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:5]
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values),'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'publisher_api': list(df_dataset['publisher_api'].iloc[book_indices].values), method_name: book_pred_score})

In [18]:
book_search = 'famous'
display(df_dataset[df_dataset.title_thalia==book_search])
print('Content-based RecSyS based on Publisher')
get_recommendations_publisher(book_search,content_correlation3, 'Predicted Rating')

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,Metadata,Topic
1,32836,famous,"jilly gagnon, todd strasser",Planet!,YFM,YXQ,"famous jilly gagnon, todd strasser Planet! YFM...",YFM YXQ


Content-based RecSyS based on Publisher


,itemID,title_thalia,publisher_api,Predicted Rating
0,26966,tripion minaccia dallo spazio,Lulu.com,0.0
1,38332,grounded at fort irwin,Xlibris Corporation,0.0
2,51507,its not ok,Xlibris Corporation,0.0
3,13238,then he ate my boy entrancers,,0.0
4,48465,startled by his furry shorts,HarperCollins UK,0.0


In [19]:
# Recommendations Based of Title
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix4 = tfidf.fit_transform(df_dataset['title_thalia'])
tfidf_df4 = pd.DataFrame(tfidf_matrix4.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [20]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation4 = linear_kernel(tfidf_matrix4, tfidf_matrix4)

In [21]:
def get_recommendations_title(title, cosine_sim, method_name):
   # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwise similarity scores of all books with that books
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the top 5 most similar movies
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:5]
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values),'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'publisher_api': list(df_dataset['publisher_api'].iloc[book_indices].values), method_name: book_pred_score})

In [22]:
book_search = 'famous'
display(df_dataset[df_dataset.title_thalia==book_search])
print('Content-based RecSyS based on Title')
get_recommendations_title(book_search,content_correlation4, 'Predicted Rating')

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,Metadata,Topic
1,32836,famous,"jilly gagnon, todd strasser",Planet!,YFM,YXQ,"famous jilly gagnon, todd strasser Planet! YFM...",YFM YXQ


Content-based RecSyS based on Title


,itemID,title_thalia,publisher_api,Predicted Rating
0,26966,tripion minaccia dallo spazio,Lulu.com,0.0
1,38332,grounded at fort irwin,Xlibris Corporation,0.0
2,51507,its not ok,Xlibris Corporation,0.0
3,13238,then he ate my boy entrancers,,0.0
4,48465,startled by his furry shorts,HarperCollins UK,0.0


In [23]:
# Recommendations Based of Title
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix5 = tfidf.fit_transform(df_dataset['Metadata'])
tfidf_df5 = pd.DataFrame(tfidf_matrix5.toarray(), index=df_dataset.index.tolist())

#count2=CountVectorizer(stop_words="english")
#count_matrix2= count.fit_transform(df_dataset['metadata'])

#count_matrix

In [24]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix based on the tfidf_matrix
content_correlation5 = linear_kernel(tfidf_matrix5, tfidf_matrix5)

In [25]:
def get_recommendations(title, cosine_sim, method_name):
   # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwise similarity scores of all books with that books
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the top 5 most similar movies
    #sim_scores = sim_scores[1:6]

    # Get relevant books
    books = [i for i in sim_scores if i[0] != idx][0:5]
    # Get the movie indices
    book_indices = [i[0] for i in books]

    # Get the pred scores
    book_pred_score = [i[1] for i in books]

    return pd.DataFrame({'itemID':  list(df_dataset['itemID'].iloc[book_indices].values),'title_thalia': list(df_dataset['title_thalia'].iloc[book_indices].values), 'Metadata': list(df_dataset['Metadata'].iloc[book_indices].values), method_name: book_pred_score})

In [26]:
book_search = 'luuurve is a many trousered thing'
display(df_dataset[df_dataset.title_thalia==book_search])
print('Content-based RecSyS based on Title')
get_recommendations(book_search,content_correlation5, 'Similarity')

,itemID,title_thalia,author_thalia,publisher_api,main topic,subtopics,Metadata,Topic
10,18472,luuurve is a many trousered thing,louise rennison,,YFM,5AM,luuurve is a many trousered thing louise renni...,YFM 5AM


Content-based RecSyS based on Title


,itemID,title_thalia,Metadata,Similarity
0,21391,stop in the name of pants,stop in the name of pants louise rennison YFM...,0.465967
1,32319,dancing in my nuddy pants,dancing in my nuddy pants louise rennison YFM...,0.432119
2,38212,knocked out by my nunga nungas,knocked out by my nunga nungas louise rennison...,0.389161
3,17850,a midsummer tights dream,a midsummer tights dream louise rennison Harpe...,0.298960
4,48465,startled by his furry shorts,startled by his furry shorts louise rennison H...,0.287690
